得益于 Python 数据模型，自定义类型行为可以像内置类型那样自然。实现如此自然的行为，靠的不是继承，而是鸭子类型，我们只需要按照预定行为实现对象所需方法即可

这一章我们定义自己的类，而且让类的行为跟真正的 Python 对象一样，这一章延续第一章，说明如何实现在很多 Python 类型中常见的特殊方法。

本章包含以下话题：

- 支持用于生成对象其他表示形式的内置函数（如 repr(), bytes() 等等）
- 使用一个类方法实现备选构造方法
- 扩展内置的 format() 函数和 str.format() 方法使用的格式微语言
- 实现只读属性
- 把对象变成可散列的，以便在集合中及作为 dict 的键使用
- 利用 `__slots__` 节省内存

我们将开发一个二维欧几里得向量模型，这个过程中覆盖上面所有话题。这个过程中我们会讨论两个概念

- 如何以及何时利用 @classmethod 和 @staticmethod 装饰器
- Python 的私有属性和受保护属性的用法，约定和局限

## 对象表示形式

每门面向对象的语言至少都有一种获取对象的字符串表示形式的标准方式。Python 提供了两种方式

repr(): 便于开发者理解的方式返回对象的字符串表示形式

str(): 便于用户理解的方式返回对象的字符串表示形式

为了给对象提供其他的表现形式，还会用到两个特殊的方法, `__bytes__` 和 `__format__`。`__bytes__` 方法与 `__str__`方法类似：bytes() 函数调用它获取对象的字节序列表示形式。而 `__format__` 方法会被内置的 format() 和 str.format() 调用。使用特殊的格式代码显示对象的字符串表示形式。

注意：Python3 中 `__repr__`, `__str__`, `__format__` 方法都必须返回 Unicode 字符串（str）类型。只有 `__bytes__` 方法应该返回字节序列（bytes 类型）

## 再谈向量类

为了说明用于生成对象表示形式的众多方法，我们将使用一个 Vector2d 类，与第一章的类似。这几节会不断完善这个类，我们期望这个类行为如下所示：

In [8]:
v1 = Vector2d(3, 4)
print(v1.x, v1.y) # 可以直接通过属性访问

3.0 4.0


In [9]:
x, y = v1 # 可以拆包成元祖
x, y

(3.0, 4.0)

In [10]:
v1

Vector2d(3.0,4.0)

In [11]:
v1_clone = eval(repr(v1)) # repr 函数调用 Vector2d 实例，结果类似于构建实例的源码
v1 == v1_clone # 支持 == 比较

True

In [12]:
print(v1) # 会调用 str 函数，对 Vector2d 来说，输出的是一个有序对

(3.0, 4.0)


In [13]:
octets = bytes(v1) # 调用 __bytes__ 方法，生成实例的二进制表示形式
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [14]:
abs(v1) # 会调用 __abs__ 方法，返回 Vector2d 实例的模

5.0

In [15]:
bool(v1), bool(Vector2d(0, 0)) # 会调用 __bool__ 方法，判断 Vector2d 的实例的向量长度

(True, False)

In [14]:
from array import array
import math

class Vector2d:
    typecode = 'd' # 类属性，Vector2d 实例和字节序之间转换使用
    
    def __init__(self, x, y):
        self.x = float(x)  # 转换成浮点数，尽早捕捉错误，防止传入不当参数
        self.y = float(y)
        
    def __iter__(self):
        return (i for i in (self.x, self.y)) # 将 Vector2d 变成可迭代对象，这样才可以拆包
    
    def __repr__(self):
        class_name = type(self).__name__
         # {!r} 获取各个分量的表示形式，然后插值，构成一个字符串。因为 Vector2d 是可迭代对象，所以用 *self 会把 x 和 y 分量提供给 format 函数
        return '{}({!r},{!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        # 我们使用 typecode 转换成字节序列然后返回
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        #为了快速比较所有分量，在操作数中构建元祖，对 Vector2d 实例来说，这样做还有问题，看下面的警告
        return tuple(self) == tuple(other) 
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))

上面的 `__eq__` 方法，在两个操作数都是 Vector2d 的时候可用，不过拿 Vector2d 实例和其他具有相同数值的可迭代对象相比，结果也是 True(如 Vector2d(3, 4) == [3, 4])。这个行为可以视为特性，也可以视为缺陷在第 13 章运算符重载时候进一步讨论

我们已经定义了许多基本方法，但是显然少了一个操作：使用 bytes() 函数生成的二进制重建 Vecotr2d 实例

## 备选构造方法

我们可以把 Vector2d 实例转成字节序列；同理，也应该能从字节序列转换成 Vector2d 实例。在标准库中探索一番之后，我们发现 array.array 有个类方法 `.frombytes`（2.91 章介绍过，从文件读取数据） 正好符合需求。下面为 Vector2d 定义一个同名的类方法

In [15]:
from array import array
import math

class Vector2d:
    typecode = 'd' 
    
    def __init__(self, x, y):
        self.x = float(x) 
        self.y = float(y)
        
    @classmethod # 类方法使用 @classmethod 装饰器
    def frombytes(cls, octets): # 不用传入 self 参数，相反，要通过 cls 传入类本身
        typecode = chr(octets[0]) #从第一个字节中读取 typecode
        # 用传入的 octets 字节序列创建一个 memoryview，然后使用 typecode 转换
        memv = memoryview(octets[1:]).cast(typecode) # 2.92 章介绍了 cast 方法，将一段内存转换成指定的类型，d 代表 float
        return cls(*memv) #拆包转换后的 memoryview，得到构造方法所需的一对参数

    def __iter__(self):
        return (i for i in (self.x, self.y)) 
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r},{!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other) 
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))

In [22]:
v1 = Vector2d(3, 4)
octets = bytes(v1)
print(octets)
v2 = Vector2d.frombytes(octets)
v2

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'


Vector2d(3.0,4.0)

我们上面用的 classmethod 装饰器是 Python 专用的，下面解释一下

## classmethod 和 staticmethod

我们来看一下 classmethod 装饰器，上面已经展示了它的用法，定义操作类，而不是操作实例的方法。classmethod 改变了调用方法的方式，因此类方法的第一个参数是类本身，而不是实例。classmethod 最常用的用途是定义备选构造方法。例如上面的 frombytes，注意，frombytes 最后一行使用 cls 参数构建了一个新实例，即 `cls(*memv)`，按照约定，类方法的第一个参数为 cls（但不是强制的）

staticmethod 装饰器也会改变方法的调用方式，但是第一个参数不是特殊值。其实，静态方法就是普通的函数，只是碰巧在类的定义体中，而不是在模块层定义。下面对这两种装饰器行为做了对比：

In [1]:
class Demo:
    @classmethod
    def klassmeth(*args):
        return args
    
    @staticmethod
    def statmeth(*args):
        return args

Demo.klassmeth()

(__main__.Demo,)

In [2]:
Demo.klassmeth('kaka')

(__main__.Demo, 'kaka')

In [3]:
Demo.statmeth()

()

In [4]:
Demo.statmeth('kaka')

('kaka',)

不管怎么调用 class.klassmeth，它的第一个参数都是 Demo 类。而 Demo.statmeth 的行为和普通函数类似。一般情况下，都是使用 classmethod，staticmethod 不是特别有用

## 格式化显示

内置的 format() 函数和  str.format() 方法把各个类型的格式化方式委托给相应的 `.__format__(format_spec)` 方法。format_spec 是格式说明符，它是：

- format(my_obj, format_spec) 的第二个参数
- str.format() 方法的格式字符串，{} 里代换字段中冒号后面的部分

In [5]:
brl =  1 / 2.43
brl

0.4115226337448559

In [6]:
format(brl, '0.4f')

'0.4115'

In [7]:
'1 BRL = {rate:0.2f} USD'.format(rate = brl)

'1 BRL = 0.41 USD'

`'{0.mass:5.3e}'` 这样的格式字符串其实包含两部分，冒号左边的 0.mass 在代换字段语法中是字段名，冒号后面的 5.3e 是格式说明符。如果对这些陌生的话，先学 format() 函数，掌握格式规范微语言，然后再阅读格式字符串语法("Format String Syntax",https://docs.python.org/3/library/string.html#formatspec)。学习 str.format() 方法使用的 {:} 代换字段表示法（包含转换标志 !s, !r, !a)

格式规范微语言为一些内置类型提供了专用的表示代码。例如，b 和 x 分别代表 二进制和十六进制的 int 类型。f 表示 float 类型，而 % 表示百分数形式

In [9]:
format(42, 'b')

'101010'

In [10]:
format(2 / 3, '.1%')

'66.7%'

格式规范微语言是可扩展的，因为各个类可以自行决定如何解释 format_spec 参数。例如，datetime 模块中的类，它们的 `__format__` 方法使用的格式代码与 strftime() 函数一样。下面是内置的 format() 函数和 str.format() 方法的几个示例：

In [11]:
from datetime import datetime
now = datetime.now()
format(now, '%H:%M:%s')

'01:02:1496941333'

In [12]:
"It's now {:%I：%M %p}".format(now)

"It's now 01：02 AM"

如果类没有定义 `__format__` 方法，从 object 继承的方法会返回 str(my_object)。我们为 Vector2d 类定义了 `__str__` 方法，因此可以这样：

In [16]:
v1 = Vector2d(3, 4)
format(v1)

'(3.0, 4.0)'

我们将实现自己的微语言解决这个问题。首先，假设用户提供的格式说明符是用于格式化向量中各个浮点数分量的。我们想达到的效果如下：

实现这种输出的 `__format__` 方法是：

In [19]:
def __format__(self, fmt_spec=''):
    components = (format(c, fmt_spec) for c in self)
    return '({}, {})'.format(*components)